##Coding
**Simulate a FrugalGPT cascade. Assume you have two models: A (cheap, 80% accuracy) and B (expensive, 90% accuracy). Define a simple confidence heuristic for model A (e.g. length of answer or presence of a certain keyword). Implement a policy that calls A, checks confidence; if confident, use A’s answer, if not, call B. Generate a dataset of queries with “ground truth” answers and simulate the cascade, measuring overall accuracy and cost. Compare this to always using B and always using A. Show how varying the confidence threshold produces a Pareto curve of cost vs. accuracy.**

frugalgpt -> if cheaper model crosses the confidence threshold then exit and use it's answer else sequentially call other models and do the same. 

2 open source models, confidence heuristic based on length of the answer. 


In [ ]:
#